In [1]:
import os
os.chdir("../")
!pwd

/home/aditya/network_security


In [ ]:
import pandas as pd

df = pd.read_csv("artifacts/data_ingestion/datafromDB.csv")

In [19]:
df.isnull().sum()

NumDots                               0
SubdomainLevel                        0
PathLevel                             0
UrlLength                             0
NumDash                               0
NumDashInHostname                     0
AtSymbol                              0
TildeSymbol                           0
NumUnderscore                         0
NumPercent                            0
NumQueryComponents                    0
NumAmpersand                          0
NumHash                               0
NumNumericChars                       0
NoHttps                               0
RandomString                          0
IpAddress                             0
DomainInSubdomains                    0
DomainInPaths                         0
HttpsInHostname                       0
HostnameLength                        0
PathLength                            0
QueryLength                           0
DoubleSlashInPath                     0
NumSensitiveWords                     0


In [13]:
from pathlib import Path
from pydantic import BaseModel

class DataTransformationConfig(BaseModel):

    ##config
    root_dir: Path
    data_path: Path
    ##params
    test_size: float
    random_state: int


In [14]:
from src.NetworkSecurity.constants import *
from src.NetworkSecurity.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:

        config = self.config.data_transformation
        params = self.params.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            test_size = params.test_size,
            random_state = params.random_state
        )

        return data_transformation_config

In [15]:
cm = ConfigurationManager()
data_transformation_config = cm.get_data_transformation_config()

[2025-03-30 22:32:26,383: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-30 22:32:26,384: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-30 22:32:26,388: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-30 22:32:26,388: INFO: common : created directory at: artifacts]
[2025-03-30 22:32:26,389: INFO: common : created directory at: artifacts/data_transformation]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.NetworkSecurity.logging.logger import logger

# perform all EDA here
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):

        data = pd.read_csv(self.config.data_path)

        train,test = train_test_split(data,
                        test_size=self.config.test_size,
                        random_state=self.config.random_state
                        )

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index= False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [17]:
dt = DataTransformation(data_transformation_config)
dt.train_test_splitting()

[2025-03-30 22:32:27,386: INFO: 2371550360 : Splitted data into training and test sets]
[2025-03-30 22:32:27,387: INFO: 2371550360 : (8000, 49)]
[2025-03-30 22:32:27,387: INFO: 2371550360 : (2000, 49)]
(8000, 49)
(2000, 49)
